## Sproj Board Scheduler

An attempt to write a script that would read faculty and students availability for the boards week, and then schedule all boards, accounting for their target composition, and student availability.

Unfortunately some of the data used by this script (names) is sensitive, and had to be stored outside of git.

In [99]:
import pandas as pd
import time
import datetime
import copy
import random
from IPython.display import HTML, display

In [100]:
hideNames = 1 # set 0 for troubleshooting, set 1 before githubbing

In [101]:
composition = pd.read_csv('../../sensitive_data/boards.csv')
students = pd.read_csv('../../sensitive_data/students.csv')
facultin = pd.read_csv('../../sensitive_data/faculty.csv')

In [102]:
# Create time grid
days = ['m','t','w','r','f']
dayName = {'m':'Mon','t':'Tue','w':'Wed','r':'Thr','f':'Fri'}
firstDate = datetime.datetime.strptime('Dec 9', "%b %d")
for _ in range(5):
    s = datetime.datetime.strftime(firstDate+datetime.timedelta(days=_),"%b %d")
    #print(s)
    dayName[days[_]] = dayName[days[_]] + ', ' + s
    # Annoying facts: 
    # 1) both time and datetime have strftime() method, but the syntax is different (sequence of arguments)
    # 2) in datetime, most useful stuff sits in datetime.datetime, but not all (timedelta doesn't)
    # 3) although both time and datetime has strptime, only one (datetime) works with datetime.timedelta

startTime = 9
endTime = 18
grid = [] # Array of tuples: day (as a letter), hour (as a 24h int number)
for d in days:
    for t in range(startTime,endTime):
        grid = grid + [(d,t)]
# print(grid[:3])

facultin.time = facultin.time.apply(lambda x: max(x,startTime))
#print(facultin)
# Note that the script doesn't check times outside of start-end window, so all ins and outs should happen either
# within this window, or before the day starts (to add or remove an entire day)

In [103]:
class Faculty:
    def __init__(self,name):
        self.name = name
        self.avail = []
        
    def __str__(self):
        return "%12s \t" % (self.name) + ''.join(['%d' % i for i in self.avail])
    
    def initAvail(self,grid,facultin):
        self.avail = [1]*len(grid)
        status = 1
        for ig in range(len(grid)):
            lin = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='in') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lin.shape[0]>0:
                status = 1
            lout = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='out') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lout.shape[0]>0:
                status = 0
            self.avail[ig] = status
            
    def updateAvail(self,grid,g,newVal=0):
        self.avail = [self.avail[i] if grid[i]!=g else newVal for i in range(len(self.avail))]
        
    def book(self,ig):
        self.avail[ig] = 0
        
        
faculty = []
for fn in composition['advisor'].append(composition['mem2']).append(composition['mem3']).unique():
    f = Faculty(fn)
    f.initAvail(grid,facultin)
    for d in days:
        f.updateAvail(grid,(d,12)) # Book lunch breaks
        if d=='w': # Wed is an advising day, so take it out completely
            for ih in range(1,25):
                f.updateAvail(grid,(d,ih),0)
    faculty.append(f)    

for f in faculty:
    print(f)

        Jude 	111011111111011111000000000000011111111011111
    Tibbetts 	000000000111011111000000000000000000000000000
   Robertson 	110001111111000000000000000111011111111011111
     Collins 	111011111111011111000000000111011111111011111
     Keesing 	111011111111011111000000000000000000000000000
   Khakhalin 	111011111111011111000000000111011111111011111
      Perron 	111011111111011111000000000111011111111011111
     Bennett 	111011111111011111000000000111011111111011111
      Dueker 	111011111111001100000000000111011111100000111
       OHara 	001011110001001000000000000011000000000000000
        Jain 	000000000000000100000000000000000100000000000
      Scalzo 	000000000000001110000000000000000100011010000


In [104]:
class Board:
    '''Board object'''
    
    def __init__(self,stuff): # Creator
        self.student = stuff['first'] + ' ' + stuff['last']
        if hideNames:
            self.student = ''.join(random.sample(self.student.lower(),len(self.student)))
        self.members = list(stuff[['advisor','mem2','mem3']])
        self.email = stuff['email']
        self.type = stuff['type']
        self.avail = []  # Placeholder: availability grid
        self.time = []
        
    def __str__(self):
        s = "%20s" % (self.student) + '\t' 
        s += ' '.join([m[:4] for m in self.members]) + '\t' # Shortened version
        s += ''.join(['%d' % i for i in self.avail])
        return s
    
    def initTimes(self,grid,students):
        '''Sets boards initial options, based on students availability'''
        data = students.loc[students.email==self.email]
        self.avail = [1]*len(grid)
        for i in range(data.shape[0]):
            h = data.iloc[i]
            if h['class'][:3]!='BIO':
                #print('Included:',[_ for _ in data.iloc[i]])
                for d in days:
                    if not pd.isna(h[d]): # Class meets on this day
                        #print(h['class'])
                        times = h['time'].split('-') # String of two times
                        for it in range(2):
                            if times[it][0]==' ':
                                times[it] = times[it][1:] # Remove leading spaces
                        tstart = time.strptime(times[0], "%I:%M %p")
                        tend   = time.strptime(times[1], "%I:%M %p")
                        for ig in range(len(grid)):
                            g = grid[ig]
                            if g[0]==d: # If this day
                                if g[1]>=tstart.tm_hour and g[1]<=tend.tm_hour: # If classes during this hour
                                    self.avail[ig] = 0
                                
    def narrowBoardTime(self):
        '''Additional requirements on some types of boards'''
        if self.type=="final": # For final boards, make Monday unavailable, as sprojes are due that day
            for ig in range(len(grid)):
                if grid[ig][0]=='m':
                    self.avail[ig] = 0
    
    def refreshFac(self,faculty):
        '''Filters boards based on current faculty availability'''
        for f in faculty:
            for facname in self.members:
                if f.name==facname:
                    self.avail = [self.avail[i]*f.avail[i] for i in range(len(self.avail))]
          
    
boards = []
print('Student availability:')
for ib in range(len(composition)):
    b = Board(composition.loc[ib,])
    b.initTimes(grid,students)
    print(b)
    boards.append(b)
    
for b in boards:
    b.narrowBoardTime()
    b.refreshFac(faculty)
    
# Rearrange from those that are harder to schedule to those that are easier
niceness = [sum(b.avail) for b in boards]
ind = [i for _,i in sorted(zip(niceness,range(len(niceness))))]
boards = [boards[i] for i in ind]

print('\nFull Board availability:')
for b in boards:
    print(b)

Student availability:
           dsiyean o	Jude Coll Kees	111111111111111111111100000111111111100111111
    qegavzeluirasb a	Tibb Jude Khak	111111111111100111111111111111100111111111111
         nfau zierad	Robe Jude Kees	111111111111111100111111111111111111000000011
        mw sralialbo	Coll Robe Kees	110001111100111111110001111100001111111111111
         ko aoertpce	Kees Coll Robe	111111111111111100111111111111111111000000011
         fdd nciehia	Khak Jude Benn	111111111100111100111111111100111100111111111
         nymocavan e	Perr Benn Khak	111111111100000111111111111111100111111111111
          brta nybne	Coll Kees Perr	100000011110001111111111111110001111111111111
       icnud bcnuoir	Benn Perr Khak	111111111111100111111111111111100111111111111
       y cmlcltihace	Kees Robe Coll	111100001100011111111100001111111111111111111
    soepnarillyot tn	Duek Kees Perr	111100011111111100111111111111111111000000011
    mal btthaizeoehs	Kees Coll Khak	1111111001001111111111000011001111111111

In [105]:
# Backup, just so that the cell below could be rerun without ruining the data
bb = copy.deepcopy(boards)
bf = copy.deepcopy(faculty)

In [106]:
boards = copy.deepcopy(bb)
faculty = copy.deepcopy(bf)

for b in boards:
    b.refreshFac(faculty)
    temp = [i for i in range(len(b.avail)) if (b.avail[i]==1)]
    if len(temp)==0:
        print('Cannot solve the puzzle for this board:')
        print(b)
        break
    ig = min(temp)
    b.time = grid[ig]
    b.avail[ig] = 0
    for f in faculty:
        if f.name in b.members:
            f.book(ig)
    print('%s\t%s\t' % (dayName[grid[ig][0]],grid[ig][1]), end='')
    print(b)
    
#for b in boards:
#    print(b)

Tue, Dec 10	9	        mw sralialbo	Coll Robe Kees	000000000000000000000000000000000000000000000
Tue, Dec 10	15	      nhnaiakrerhc h	Perr Duek Jain	000000000000000000000000000000000100000000000
Thr, Dec 12	15	    apeeanjtu sewene	Perr Benn Jain	000000000000000000000000000000000000000000000
Tue, Dec 10	10	         nfau zierad	Robe Jude Kees	000000000001000000000000000000000000000000000
Tue, Dec 10	11	         ko aoertpce	Kees Coll Robe	000000000000000000000000000000000000000000000
Mon, Dec 09	9	       y cmlcltihace	Kees Robe Coll	010000001000000000000000000000000000000000000
Tue, Dec 10	9	    qegavzeluirasb a	Tibb Jude Khak	000000000001000111000000000000000000000000000
Tue, Dec 10	13	         yeeetdgnlnc	Jude Tibb Coll	000000000000001111000000000000000000000000000
Mon, Dec 09	11	ilr-iao pneztpefholek	Kees OHar Khak	000011110000001000000000000000000000000000000
Tue, Dec 10	14	  armaamtlacseealatn	Perr Jude Scal	000000000000000010000000000000000000011010000
Tue, Dec 10	15	           dsiyea

In [107]:
ind = [min([i for i in range(len(grid)) if grid[i]==b.time]) for b in boards]
ind = [j for (i,j) in sorted(zip(ind,range(len(ind))))]
boards = [boards[i] for i in ind]

def usTime(time):
    if time>12:
        return ("%dp" % (time-12))
    else:
        return ("%da" % (time))

def printboard(b,mode="none"):
    if mode=="none":
        print("%22s\t%12s\t%s\t%d\t" % (b.student,b.type,dayName[b.time[0]],b.time[1]),end='')
        for fn in b.members:
            print("%10s\t" % (fn),end='')
        print()
    elif mode=="tabs":
        if not hideNames:
            print("%s\t%s\t%s\t%s\t%s\t" % (b.student,b.email,b.type,dayName[b.time[0]],usTime(b.time[1])),end='')
        else:
            print("%s\t%s\t%s\t%s\t%s\t" % (b.student,"nope@nope.edu",b.type,dayName[b.time[0]],usTime(b.time[1])),end='')
        for fn in b.members:
            print("%s\t" % (fn),end='')
        print()
    elif mode=="html":
        s = ''
        s += "<tr><td>%s</td><td>%s</td><td>%s %d</td><td>" % (b.student,b.type,dayName[b.time[0]],b.time[1])
        for fn in b.members:
            s += "%s " % (fn)
        s += "</td></tr>"
        display(HTML(s))

for b in boards:
    printboard(b,mode="tabs")

y cmlcltihace	nope@nope.edu	midway	Mon, Dec 09	9a	Keesing	Robertson	Collins	
sitaqnraa 	nope@nope.edu	midway	Mon, Dec 09	9a	Khakhalin	Jude	Bennett	
soepnarillyot tn	nope@nope.edu	midway	Mon, Dec 09	10a	Dueker	Keesing	Perron	
fdd nciehia	nope@nope.edu	midway	Mon, Dec 09	10a	Khakhalin	Jude	Bennett	
ilr-iao pneztpefholek	nope@nope.edu	moderation	Mon, Dec 09	11a	Keesing	OHara	Khakhalin	
mljt sehiwe	nope@nope.edu	midway	Mon, Dec 09	11a	Collins	Jude	Bennett	
eorsmn kae	nope@nope.edu	moderation	Mon, Dec 09	1p	Keesing	Jude	Khakhalin	
mal btthaizeoehs	nope@nope.edu	midway	Mon, Dec 09	2p	Keesing	Collins	Khakhalin	
dayn anromzd	nope@nope.edu	midway	Mon, Dec 09	2p	Robertson	Perron	Dueker	
islra dsualen	nope@nope.edu	midway	Mon, Dec 09	3p	Keesing	Collins	Perron	
cdrmohre calaie	nope@nope.edu	midway	Mon, Dec 09	3p	Khakhalin	Jude	Bennett	
brta nybne	nope@nope.edu	midway	Mon, Dec 09	4p	Collins	Keesing	Perron	
ahbtbg rmynaa	nope@nope.edu	midway	Mon, Dec 09	5p	Robertson	Perron	Dueker	
 yreaaclmngei	nope

In [108]:
#Sort by faculty:

for f in faculty:
    print()
    print(f.name)
    for b in boards:
        if f.name in b.members:
            printboard(b,mode="tabs")


Jude
sitaqnraa 	nope@nope.edu	midway	Mon, Dec 09	9a	Khakhalin	Jude	Bennett	
fdd nciehia	nope@nope.edu	midway	Mon, Dec 09	10a	Khakhalin	Jude	Bennett	
mljt sehiwe	nope@nope.edu	midway	Mon, Dec 09	11a	Collins	Jude	Bennett	
eorsmn kae	nope@nope.edu	moderation	Mon, Dec 09	1p	Keesing	Jude	Khakhalin	
cdrmohre calaie	nope@nope.edu	midway	Mon, Dec 09	3p	Khakhalin	Jude	Bennett	
 yreaaclmngei	nope@nope.edu	moderation	Mon, Dec 09	5p	Keesing	Jude	Khakhalin	
qegavzeluirasb a	nope@nope.edu	final	Tue, Dec 10	9a	Tibbetts	Jude	Khakhalin	
nfau zierad	nope@nope.edu	final	Tue, Dec 10	10a	Robertson	Jude	Keesing	
ullaehmeci h	nope@nope.edu	midway	Tue, Dec 10	11a	Jude	Tibbetts	Khakhalin	
 yeeetdgnlnc	nope@nope.edu	midway	Tue, Dec 10	1p	Jude	Tibbetts	Collins	
 armaamtlacseealatn	nope@nope.edu	midway	Tue, Dec 10	2p	Perron	Jude	Scalzo	
dsiyean o	nope@nope.edu	final	Tue, Dec 10	3p	Jude	Collins	Keesing	
ijin lgubaso	nope@nope.edu	midway	Tue, Dec 10	4p	Jude	Keesing	Tibbetts	

Tibbetts
qegavzeluirasb a	nope@nope.ed